In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

model = VGG16(weights='imagenet', include_top=True)
target_size = (224, 224)

def resize_images(images, size):
    resized_images = np.zeros((images.shape[0], size[0], size[1], 3))
    for i in range(images.shape[0]):
        resized_images[i] = tf.image.resize(images[i], size).numpy()
    return resized_images

def preprocess_images(images):
    return preprocess_input(images)

cifar10_labels_to_imagenet_labels = {
    0: 404,  # airplane -> airplane (as similar as possible)
    1: 817,  # automobile -> sports car
    2: 386,  # bird -> bird
    3: 281,  # cat -> tabby cat
    4: 345,  # deer -> deer
    5: 207,  # dog -> golden retriever
    6: 30,   # frog -> tree frog
    7: 515,  # horse -> horse
    8: 625,  # ship -> warship
    9: 867   # truck -> tow truck
}

train_labels_imagenet = np.array([cifar10_labels_to_imagenet_labels[label[0]] for label in train_labels])
test_labels_imagenet = np.array([cifar10_labels_to_imagenet_labels[label[0]] for label in test_labels])

train_labels_imagenet = to_categorical(train_labels_imagenet, 1000)
test_labels_imagenet = to_categorical(test_labels_imagenet, 1000)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

batch_size = 100
num_batches = len(test_images) // batch_size

accuracy_list = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    batch_images = resize_images(test_images[start_idx:end_idx], target_size)
    batch_images_preprocessed = preprocess_images(batch_images)
    batch_labels = test_labels_imagenet[start_idx:end_idx]
    
    loss, accuracy = model.evaluate(batch_images_preprocessed, batch_labels, verbose=0)
    accuracy_list.append(accuracy)

average_accuracy = np.mean(accuracy_list)
print(f'Average accuracy of VGG16 model on CIFAR-10 dataset: {average_accuracy * 100:.2f}%')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import numpy as np
from tensorflow.keras.datasets import cifar10
from keras.regularizers import l2

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

target_size = (128, 128)
batch_size = 32

cifar10_labels_to_imagenet_labels = {
    0: 404,  # airplane -> airplane (as similar as possible)
    1: 817,  # automobile -> sports car
    2: 386,  # bird -> bird
    3: 281,  # cat -> tabby cat
    4: 345,  # deer -> deer
    5: 207,  # dog -> golden retriever
    6: 30,   # frog -> tree frog
    7: 515,  # horse -> horse
    8: 625,  # ship -> warship
    9: 867   # truck -> tow truck
}

train_labels_imagenet = np.array([cifar10_labels_to_imagenet_labels[label[0]] for label in train_labels])
test_labels_imagenet = np.array([cifar10_labels_to_imagenet_labels[label[0]] for label in test_labels])

train_labels_imagenet = to_categorical(train_labels_imagenet, 1000)
test_labels_imagenet = to_categorical(test_labels_imagenet, 1000)

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow(
    x=train_images,
    y=train_labels_imagenet,
    batch_size=batch_size,
    shuffle=True
)

test_generator = test_datagen.flow(
    x=test_images,
    y=test_labels_imagenet,
    batch_size=batch_size,
    shuffle=False
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))

for layer in base_model.layers[:-4]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
predictions = Dense(1000, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)

model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[checkpoint, reduce_lr],
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)

model.load_weights('best_model.h5')

loss, accuracy = model.evaluate(test_generator)
print(f'Fine-tuned VGG16 model accuracy on CIFAR-10 dataset: {accuracy * 100:.2f}%')